In [21]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [22]:
merged_data=pd.read_csv(r'C:\Users\augus\OneDrive\Desktop\js\Cargo predict\import\dataset\combined_set.csv')
merged_data

,Sail Date.,QTY.(KGS)
0,2018-01-01,257733
1,2018-01-04,577207
2,2018-01-05,1322201
3,2018-01-06,1652038
4,2018-01-07,1548883
...,...,...
1333,2022-12-27,1029546
1334,2022-12-28,971387
1335,2022-12-29,231274
1336,2022-12-30,358590


In [23]:
# Split the data into features (X) and target variable (y)
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Assuming 'merged_data' is your DataFrame
# merged_data = pd.read_csv('your_file.csv')  # Read your data from a CSV file or any other source

# Convert 'Sail Date.' to datetime format
merged_data['Sail Date.'] = pd.to_datetime(merged_data['Sail Date.'])

# Extract year, month, and day as numerical features
merged_data['Year'] = merged_data['Sail Date.'].dt.year
merged_data['Month'] = merged_data['Sail Date.'].dt.month
merged_data['Day'] = merged_data['Sail Date.'].dt.day

# Drop the original 'Sail Date.' column
merged_data.drop(columns=['Sail Date.'], inplace=True)

# Split the data into features (X) and target variable (y)
X = merged_data[['Year', 'Month', 'Day']]  # Features
y = merged_data['QTY.(KGS)']                # Target variable


# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


X.shape




(1338, 3)

In [24]:

import pandas as pd
from sklearn.model_selection import train_test_split

# Assuming 'merged_data' is your DataFrame with columns 'Year', 'Month', 'Day', and 'QTY.(KGS)'
# You can replace 'merged_data' with your actual DataFrame

# Filter data for 2018 to 2021
train_data = merged_data[(merged_data['Year'] == 2018) | (merged_data['Year'] == 2019) | (merged_data['Year'] == 2020) | (merged_data['Year'] == 2021)  ]

# Filter data for 2022
test_data = merged_data[(merged_data['Year'] == 2022)]

# Split train_data into features (X_train) and target variable (y_train)
X_train = train_data[['Year', 'Month', 'Day']]
y_train = train_data['QTY.(KGS)']

# Split test_data into features (X_test) and target variable (y_test)
X_test = test_data[['Year', 'Month', 'Day']]
y_test = test_data['QTY.(KGS)']

# Optionally, you can reset the indices of the DataFrames
X_train.reset_index(drop=True, inplace=True)
y_train.reset_index(drop=True, inplace=True)
X_test.reset_index(drop=True, inplace=True)
y_test.reset_index(drop=True, inplace=True)

# Print the shapes of the train and test sets
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)


X_train shape: (1074, 3)
y_train shape: (1074,)
X_test shape: (264, 3)
y_test shape: (264,)


In [25]:
X_train

,Year,Month,Day
0,2018,1,1
1,2018,1,4
2,2018,1,5
3,2018,1,6
4,2018,1,7
...,...,...,...
1069,2021,12,24
1070,2021,12,25
1071,2021,12,27
1072,2021,12,29


In [26]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Assuming X_train, y_train, X_test are pandas DataFrames

# Convert DataFrame columns to numpy arrays
X_train_array = X_train.to_numpy()
y_train_array = y_train.to_numpy()
X_test_array = X_test.to_numpy()

# Reshape the input data for LSTM (assuming 3D arrays)
X_train_reshaped = X_train_array.reshape((X_train_array.shape[0], X_train_array.shape[1], 1))
X_test_reshaped = X_test_array.reshape((X_test_array.shape[0], X_test_array.shape[1], 1))

# Build the LSTM model
model = Sequential([
    LSTM(units=50, input_shape=(X_train_reshaped.shape[1], 1)),
    Dense(units=1)
])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X_train_reshaped, y_train_array, epochs=100, batch_size=32)

# Predict for every date in 2022
predicted_qty = model.predict(X_test_reshaped)

# Print or use the predicted_qty for further analysis
print(predicted_qty)


Epoch 1/100
34/34 [==============================] - 3s 5ms/step - loss: 1204264239104.0000
Epoch 2/100
34/34 [==============================] - 0s 4ms/step - loss: 1204259127296.0000
Epoch 3/100
34/34 [==============================] - 0s 5ms/step - loss: 1204251394048.0000
Epoch 4/100
34/34 [==============================] - 0s 6ms/step - loss: 1204244578304.0000
Epoch 5/100
34/34 [==============================] - 0s 6ms/step - loss: 1204239597568.0000
Epoch 6/100
34/34 [==============================] - 0s 4ms/step - loss: 1204235272192.0000
Epoch 7/100
34/34 [==============================] - 0s 6ms/step - loss: 1204231733248.0000
Epoch 8/100
34/34 [==============================] - 0s 6ms/step - loss: 1204227932160.0000
Epoch 9/100
34/34 [==============================] - 0s 5ms/step - loss: 1204224393216.0000
Epoch 10/100
34/34 [==============================] - 0s 4ms/step - loss: 1204220854272.0000
Epoch 11/100
34/34 [==============================] - 0s 6ms/step - loss: 12042

In [27]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Calculate Mean Squared Error
mse = mean_squared_error(y_test, predicted_qty)
print("Mean Squared Error:", mse)

# Calculate Mean Absolute Error
mae = mean_absolute_error(y_test, predicted_qty)
print("Mean Absolute Error:", mae)


Mean Squared Error: 1510121826177.1042
Mean Absolute Error: 987529.1284547285


In [28]:
rmse = np.sqrt(mse)
print("Root Mean Squared Error:", rmse)


Root Mean Squared Error: 1228870.1421131138
